In [2]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

In [2]:
import sys
print("Python version is "+sys.version)
from keras import __version__
print(__version__)

Python version is 3.5.2 (default, Nov 17 2016, 17:05:23) 
[GCC 5.4.0 20160609]
2.0.4


In [3]:
X_train = np.array([[[[1,1],
                     [1,1]],
                     [[1,1],
                     [1,1]],
                     [[1,1],
                     [1,1]] ],
                    [[[2,2],
                     [2,2]],
                     [[2,2],
                     [2,2]],
                     [[2,2],
                     [2,2]] ] ]) # theano

X_train = X_train.reshape(2,2,2,3)


Y_train = np.array([1,2])

In [4]:
print(X_train.shape)
X_train

(2, 2, 2, 3)


array([[[[1, 1, 1],
         [1, 1, 1]],

        [[1, 1, 1],
         [1, 1, 1]]],


       [[[2, 2, 2],
         [2, 2, 2]],

        [[2, 2, 2],
         [2, 2, 2]]]])

In [5]:
train_datagen = ImageDataGenerator()

keras_example_iterator = train_datagen.flow(X_train, Y_train, batch_size=1, shuffle=False)

In [6]:
keras_example_iterator.next()

(array([[[[ 1.,  1.,  1.],
          [ 1.,  1.,  1.]],
 
         [[ 1.,  1.,  1.],
          [ 1.,  1.,  1.]]]], dtype=float32), array([1]))

In [7]:
keras_example_iterator.next()

(array([[[[ 2.,  2.,  2.],
          [ 2.,  2.,  2.]],
 
         [[ 2.,  2.,  2.],
          [ 2.,  2.,  2.]]]], dtype=float32), array([2]))

In [8]:
keras_example_iterator.next() # goes back to the start (loops over the data indefinitely - as per the doc (run a "Find in page" for `fit_generator` https://keras.io/models/model/

(array([[[[ 1.,  1.,  1.],
          [ 1.,  1.,  1.]],
 
         [[ 1.,  1.,  1.],
          [ 1.,  1.,  1.]]]], dtype=float32), array([1]))

Lets add a validation set to creating a generator for pseudo-labelling.

In [4]:
X_val = np.array([[[[3,3],
                     [3,3]],
                     [[3,3],
                     [3,3]],
                     [[3,3],
                     [3,3]] ],
                    [[[4,4],
                     [4,4]],
                     [[4,4],
                     [4,4]],
                     [[4,4],
                     [4,4]] ],
                     [[[5,5],
                     [5,5]],
                     [[5,5],
                     [5,5]],
                     [[5,5],
                     [5,5]] ]]) # theano

X_val = X_val.reshape(3,2,2,3)


pseudo_Y_val = np.array([3,4,5])

In [5]:
train_datagen = ImageDataGenerator()
val_datagen = train_datagen  # "don't forget to perform any training preprocessing on the val/test set
                             # (not neccessarily the augmentation though)." 

train_iter = train_datagen.flow(X_train, Y_train, batch_size=2, shuffle=False)
val_iter = val_datagen.flow(X_val, pseudo_Y_val, batch_size=1, shuffle=False)

In [11]:
print(train_iter.next())
print(val_iter.next())

(array([[[[ 1.,  1.,  1.],
         [ 1.,  1.,  1.]],

        [[ 1.,  1.,  1.],
         [ 1.,  1.,  1.]]],


       [[[ 2.,  2.,  2.],
         [ 2.,  2.,  2.]],

        [[ 2.,  2.,  2.],
         [ 2.,  2.,  2.]]]], dtype=float32), array([1, 2]))
(array([[[[ 3.,  3.,  3.],
         [ 3.,  3.,  3.]],

        [[ 3.,  3.,  3.],
         [ 3.,  3.,  3.]]]], dtype=float32), array([3]))


In [12]:
print(train_iter.next())
print(val_iter.next())

(array([[[[ 1.,  1.,  1.],
         [ 1.,  1.,  1.]],

        [[ 1.,  1.,  1.],
         [ 1.,  1.,  1.]]],


       [[[ 2.,  2.,  2.],
         [ 2.,  2.,  2.]],

        [[ 2.,  2.,  2.],
         [ 2.,  2.,  2.]]]], dtype=float32), array([1, 2]))
(array([[[[ 4.,  4.,  4.],
         [ 4.,  4.,  4.]],

        [[ 4.,  4.,  4.],
         [ 4.,  4.,  4.]]]], dtype=float32), array([4]))


In [13]:
print(train_iter.next())
print(val_iter.next())

(array([[[[ 1.,  1.,  1.],
         [ 1.,  1.,  1.]],

        [[ 1.,  1.,  1.],
         [ 1.,  1.,  1.]]],


       [[[ 2.,  2.,  2.],
         [ 2.,  2.,  2.]],

        [[ 2.,  2.,  2.],
         [ 2.,  2.,  2.]]]], dtype=float32), array([1, 2]))
(array([[[[ 5.,  5.,  5.],
         [ 5.,  5.,  5.]],

        [[ 5.,  5.,  5.],
         [ 5.,  5.,  5.]]]], dtype=float32), array([5]))


In [7]:
class WeightedIterator(object):
    # Based on JH `MixIterator` https://github.com/asmith26/courses/blob/master/deeplearning1/nbs/utils.py#L237-L262
    def __init__(self, iters):
        assert type(iters) is list, "Initialising a WeightedIterator requires one *list* argument."
        self.iters = iters
        self.n = sum([it.n for it in self.iters])

    def reset(self):
        for it in self.iters: it.reset()

    def __iter__(self):
        return self
    
    def __next__(self, *args, **kwargs):
        return self.next(*args, **kwargs)

    def next(self, *args, **kwargs):
        """For python 2.x.

        # Returns
            The next batch.
        """
        next_iters_batches = [it.next() for it in self.iters]
        next_batch_X = np.concatenate([next_iter_batch[0] for next_iter_batch in next_iters_batches])
        next_batch_Y = np.concatenate([next_iter_batch[1] for next_iter_batch in next_iters_batches])
        return next_batch_X, next_batch_Y

In [15]:
weighted_iter = WeightedIterator([train_iter, val_iter])

In [31]:
print(weighted_iter.next())

(array([[[[ 1.,  1.,  1.],
         [ 1.,  1.,  1.]],

        [[ 1.,  1.,  1.],
         [ 1.,  1.,  1.]]],


       [[[ 2.,  2.,  2.],
         [ 2.,  2.,  2.]],

        [[ 2.,  2.,  2.],
         [ 2.,  2.,  2.]]],


       [[[ 3.,  3.,  3.],
         [ 3.,  3.,  3.]],

        [[ 3.,  3.,  3.],
         [ 3.,  3.,  3.]]]], dtype=float32), array([1, 2, 3]))


In [32]:
print(weighted_iter.next())

(array([[[[ 1.,  1.,  1.],
         [ 1.,  1.,  1.]],

        [[ 1.,  1.,  1.],
         [ 1.,  1.,  1.]]],


       [[[ 2.,  2.,  2.],
         [ 2.,  2.,  2.]],

        [[ 2.,  2.,  2.],
         [ 2.,  2.,  2.]]],


       [[[ 4.,  4.,  4.],
         [ 4.,  4.,  4.]],

        [[ 4.,  4.,  4.],
         [ 4.,  4.,  4.]]]], dtype=float32), array([1, 2, 4]))


In [33]:
print(weighted_iter.next())

(array([[[[ 1.,  1.,  1.],
         [ 1.,  1.,  1.]],

        [[ 1.,  1.,  1.],
         [ 1.,  1.,  1.]]],


       [[[ 2.,  2.,  2.],
         [ 2.,  2.,  2.]],

        [[ 2.,  2.,  2.],
         [ 2.,  2.,  2.]]],


       [[[ 5.,  5.,  5.],
         [ 5.,  5.,  5.]],

        [[ 5.,  5.,  5.],
         [ 5.,  5.,  5.]]]], dtype=float32), array([1, 2, 5]))


In [34]:
print(weighted_iter.next())

(array([[[[ 1.,  1.,  1.],
         [ 1.,  1.,  1.]],

        [[ 1.,  1.,  1.],
         [ 1.,  1.,  1.]]],


       [[[ 2.,  2.,  2.],
         [ 2.,  2.,  2.]],

        [[ 2.,  2.,  2.],
         [ 2.,  2.,  2.]]],


       [[[ 3.,  3.,  3.],
         [ 3.,  3.,  3.]],

        [[ 3.,  3.,  3.],
         [ 3.,  3.,  3.]]]], dtype=float32), array([1, 2, 3]))


Example training:

In [9]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential([
    Dense(12, input_shape=(2,2,3)),
    Dense(1)
])
model.compile(optimizer='sgd',
              loss='mean_squared_error')

In [10]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 2, 2, 12)          48        
_________________________________________________________________
dense_2 (Dense)              (None, 2, 2, 1)           13        
Total params: 61
Trainable params: 61
Non-trainable params: 0
_________________________________________________________________


In [1]:
model.fit_generator(generator=weighted_iter,
                    steps_per_epoch=3,
                    nb_epoch=8)

NameError: name 'model' is not defined